PPCA lavora con più variabili osservabili anzichè una sola

In [1]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
scaler = StandardScaler()

In [3]:
data = np.load('prova.npy', allow_pickle=True).item()

# extract data social anxiety

In [4]:
import pandas as pd

In [5]:
# csv_ = '/Users/marcoghezzi/PycharmProjects/pythonProject/osfstorage-archive/behavior/LookAtMe_002.csv'
csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/Discrimination_LookAtMe_002.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d>2) for d in global_data['rating']]))
e_labels = y[:,np.newaxis]  # rating > 2

In [6]:
N_e = e_labels.shape[0]
D_e = e_labels.shape[1]
print(N_e,D_e)

40 1


## test with new data

In [7]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets
import deepemogp.signal.behavior as behavior

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(2, 1))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f2)
hr_ = physio.HR(f2)

pupil_ = behavior.PUPIL(f2)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_}, subjects={'2_69'})

for s in d.signals:
    # preprocess ...
    #s.preprocess(show=show)

    # ... and extract features from each signal type
    s.feature_ext.extract_feat_without_preprocess(s, show=show)

hr_data = list(d.signals)[0].features
pupil_data = list(d.signals)[1].features

>> Loading HR for subject 2 and session 69 from dataset FEAR
>> Loading PUPIL for subject 2 and session 69 from dataset FEAR
>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
params: 
1000
params: 
1000
params: 
1000
params: 
1000
params: 
1000
>> Extracting wavelet features from PUPIL signal, adopting (2, 1) window ...
params: 
200
params: 
200
params: 
200
params: 
200


In [8]:
hr_raw = list(d.signals)[0].raw[0]['data']

In [9]:
d.signals

{<deepemogp.signal.physio.hr.HR at 0x7fc841ca3640>,
 <deepemogp.signal.behavior.pupil.PUPIL at 0x7fc840618c70>}

In [10]:
hr_data

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.214719,-0.969489,-1.794818,-0.130028,0.648658,-1.278176,0.655360,-0.065799,-0.345131,0.717082,-0.309098,-0.737570
1,2.978765,3.028443,3.454952,1.468468,-0.603029,0.742600,-0.289749,-0.124492,0.737946,-0.706377,0.599640,-1.646102
2,-0.786910,-0.700728,-0.907563,-0.142147,0.700752,-0.470463,-0.046469,0.712655,-0.892832,-0.011727,0.577033,-1.191729
3,-1.539465,-1.560444,-1.752965,-0.468737,-0.789217,-0.077583,0.669867,-1.193513,0.529042,0.386148,-0.665355,-1.387225
4,-1.208558,-1.196978,-1.151932,-1.304946,0.252191,0.745649,-1.251371,0.803481,0.048910,-0.673520,0.016190,2.805684


In [11]:
pupil_data

,0,1,2,3,4,5
0,24.539762,24.532551,24.499755,24.704881,24.280770,23.631410
1,24.246797,24.234859,24.261076,24.119206,24.118721,23.966737
2,23.709426,23.689816,23.623407,23.988737,23.761592,23.567655
3,24.065313,24.064917,24.120272,23.714979,24.327230,24.581417


## end test

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variability
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

gaze =

In [12]:
pupil = pupil_data
hr = hr_data
#eda = data['eda']
'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


"social_anx = data['social anxiety'][:,np.newaxis]\npain_exp = data['pain expectation'][:,np.newaxis]"

In [13]:
print(pupil.shape)
print(hr.shape)

(4, 6)
(5, 12)


In [14]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

#N_eda = eda.shape[0]
#D_eda = eda.shape[1]
K = 1

print(N_pupil,D_pupil)
print(N_hr,D_hr)

4 6
5 12


In [19]:
with pm.Model() as sPPCA:
    #dati osservabili
    pupil_data = pm.MutableData("pupil_data", pupil.T)
    hr_data = pm.MutableData("hr_data", hr.T)
    #eda_data = pm.MutableData("eda_data", eda.T)

    #matrici pesi
    Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    Wexp = pm.Normal('W e', mu = at.zeros([D_e,K]), sigma=2.0 * at.ones([D_e, K]),shape=[D_e,K])


    c = pm.Normal('c', mu=at.zeros([N_pupil, K]), sigma=at.ones([N_pupil, K]), shape=[N_pupil, K])

    # dati della dilatazione pupille interpretati come una gaussiana
    x_pupil = pm.Normal('x_pupil', mu=Wpupil.dot(c.T), sigma=at.ones([D_pupil, N_pupil]), shape=[D_pupil, N_pupil], observed=pupil_data)

    # dati dell'hrv interpretati come una gaussiana
    x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)

    #latent space

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    pain_expectation = pm.Bernoulli('pain_exp' , p=pm.math.sigmoid(Wexp.dot(c.T)) , shape =[N_e,K])


    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [20]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

'.gv.pdf'

In [17]:
sPPCA.free_RVs

[Wpupil ~ N(f(), f()),
 Whr ~ N(f(), f()),
 W e ~ N(f(), f()),
 c ~ N(f(), f()),
 pain_exp ~ Bern(f(W e, c))]

In [18]:
with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,progressbar=True)

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

ValueError: could not broadcast input array from shape (1,40) into shape (40,1)
Apply node that caused the error: Alloc(TensorConstant{(1, 40) of 0.5}, TensorConstant{40}, TensorConstant{1})
Toposort index: 4
Inputs types: [TensorType(float64, (1, 40)), TensorType(int64, ()), TensorType(int64, ())]
Inputs shapes: [(1, 40), (), ()]
Inputs strides: [(320, 8), (), ()]
Inputs values: ['not shown', array(40), array(1)]
Outputs clients: [[Elemwise{lt,no_inplace}(Alloc.0, TensorConstant{(1, 1) of 0.5})]]

Backtrace when the node is created (use Aesara flag traceback__limit=N to make it longer):
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/pymc/model.py", line 638, in logp_dlogp_function
    ip = self.initial_point(0)
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/pymc/model.py", line 1068, in initial_point
    fn = make_initial_point_fn(model=self, return_transformed=True)
  File "/home/paolo/matteo/ma

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"gaze_data": pupil})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
hr_pred = posterior_predictive.posterior_predictive["x_hr"]

In [ ]:
pupil

In [ ]:
print(hr_pred[0])

In [ ]:
print(hr_pred[0].shape)

In [ ]:
print(float(np.mean(hr_pred[0])))

median abs deviation prb is not the optimal solution

In [ ]:
hr_pred_mean = (stats.median_abs_deviation(hr_pred[0]))[0]
hr_pred_mean

In [ ]:
hr[0]

In [ ]:
#hr_pred_mode= pd.Series([0.006,0.01,0.02,0.003])

hr and hr_pred_mode devono avere same shape

In [ ]:
train_accuracy_hr = (np.subtract(hr[0], hr_pred_mean))
print('\n\t avg distance hr: \n' + str(train_accuracy_hr))
#print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
    gaze_pred = posterior_predictive.posterior_predictive["x_gaze"]

In [ ]:
gaze_pred

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = pupil.reshape(-1)
reconstructed_r2 = metrics.r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)